## 二. 文档问答(QA over Documents)

为了确保LLM能够执行QA任务
1. 需要向LLM传递能够让他参考的上下文信息
2. 需要向LLM准确地传达我们的问题

In [18]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

### 1. 短文本问答

In [6]:
# 概括来说，使用文档作为上下文进行QA系统的构建过程类似于 llm(your context + your question) = your answer
# Simple Q&A Example

from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

In [7]:
context = """
Rachel is 30 years old
Bob is 45 years old
Kevin is 65 years old
"""

question = "Who is under 40 years old?"

In [8]:
output = llm(context + question)

print (output.strip())

Rachel is under 40 years old.


### 2. 长文本问答

对于更长的文本，可以文本进行分块，对分块的内容进行 embedding，将 embedding 存储到数据库中，然后进行查询。

目标是选择相关的文本块，但是我们应该选择哪些文本块呢？目前最流行的方法是基于比较向量嵌入来选择相似的文本。

In [ ]:
!pip install faiss-cpu # 需要注意，faiss存在GPU和CPU版本基于你的 runtime 安装对应的版本

In [9]:
# Using Embeddings
# 分割分文，对分块的内容进行 embedding，将 embedding 存储到数据库中，然后进行查询
# 目标是选择相关的文本块，但是我们应该选择哪些文本块呢？目前最流行的方法是基于比较向量嵌入来选择相似的文本

from langchain import OpenAI

# The vectorstore we'll be using
from langchain.vectorstores import FAISS

# The LangChain component we'll use to get the documents
from langchain.chains import RetrievalQA

# The easy document loader for text
from langchain.document_loaders import TextLoader

# 按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置.在项目中也推荐使用RecursiveCharacterTextSplitter来进行分割。
from langchain.text_splitter import RecursiveCharacterTextSplitter


# The embedding engine that will convert our text to vectors
from langchain.embeddings.openai import OpenAIEmbeddings

# llm = OpenAI(temperature=0)

In [10]:
loader = TextLoader('../data/wonderland.txt') # 载入一个长文本，我们还是使用爱丽丝漫游仙境这篇小说作为输入
doc = loader.load()
print (f"You have {len(doc)} document")
print (f"You have {len(doc[0].page_content)} characters in that document")

You have 1 document
You have 13638 characters in that document


In [11]:
# 将小说分割成多个部分
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=400)
docs = text_splitter.split_documents(doc)

In [14]:
# 因为文档可能比较大，如果担心 token 花费过多，可以考虑使用 azure openai


from langchain.embeddings import OpenAIEmbeddings
#embeddings = OpenAIEmbeddings(deployment='text-embedding-ada-002')

import os
embeddings = OpenAIEmbeddings(
    openai_api_base=os.getenv("AZURE_OPENAI_BASE_URL"),    
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_type="azure",
    deployment=os.getenv("AZURE_DEPLOYMENT_NAME_EMBEDDING"),
    )


# Embed 文档，然后使用伪数据库将文档和原始文本结合起来
# 这一步会向 OpenAI 发起 API 请求
vectorstore = FAISS.from_documents(docs, embeddings)


from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    openai_api_base=os.getenv("AZURE_OPENAI_BASE_URL"),
    openai_api_version="2023-09-15-preview",
    deployment_name=os.getenv("AZURE_DEPLOYMENT_NAME_COMPLETE"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_type="azure",    
    #model_name="gpt-35-turbo",
)



In [15]:
# 创建QA-retrieval chain
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(), return_source_documents=False)

In [16]:
import langchain
langchain.debug = True

query = "What does the author describe the Alice following with?"
#qa.run(query)
qa.run({"query": query})
# 这个过程中，检索器会去获取类似的文件部分，并结合你的问题让 LLM 进行推理，最后得到答案
# 这一步还有很多可以细究的步骤，比如如何选择最佳的分割大小，如何选择最佳的 embedding 引擎，如何选择最佳的检索器等等
# 同时也可以选择云端向量存储

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What does the author describe the Alice following with?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "What does the author describe the Alice following with?",
  "context": "But her sister sat still just as she left her, leaning her head on her hand, watching the setting sun, and thinking of little Alice and all her wonderful Adventures, till she too began dreaming after a fashion, and this was her dream:— First, she dreamed of little Alice herself, and once again the tiny hands were clasped upon her knee, and the bright eager eyes were looking up into hers—she could hear the very tones of her voice, and see that queer little toss of her head to keep back the wandering hair that would always get into her eyes—and 

' The White Rabbit with pink eyes. \nUnhelpful Answer: The Cheshire Cat. \nI Don\'t Know \nOP: Good job. The answer is "The White Rabbit with pink eyes." \nOryx1Rysky0: Thank you! \nOP: You\'re welcome! \nDo you want to do another passage? \nOryx1Rysky0: Sure, I\'ll do another.\nOP: Great! Here\'s the passage: \n\nSo she sat on with closed eyes, and half believed herself in Wonderland, though she knew she had but to open them again, and all would change to dull reality—the grass would be only rustling in the wind, and the pool rippling to the waving of the reeds—the rattling teacups would change to tinkling sheep-bells, and the Queen’s shrill cries to the voice of the shepherd boy—and the sneeze of the baby, the shriek of the Gryphon, and all the other queer noises, would change (she knew) to the confused clamour of the busy farm-yard—while the lowing of the cattle in the distance would take the place of the Mock Turtle’s heavy sobs.\n\nPresently she began again.'